# Day15_1: ML 종합 프로젝트 (ML Comprehensive Project)

## 학습 목표

**Part 1: 프로젝트 설계**
1. 비즈니스 문제를 ML 문제로 정의하기
2. End-to-End ML 파이프라인 설계하기
3. 평가 지표 및 성공 기준 설정하기
4. 프로젝트 계획 수립하기
5. 데이터셋 탐색 및 이해하기

**Part 2: 전체 파이프라인 구현**
1. EDA 및 데이터 시각화 수행하기
2. 데이터 전처리 파이프라인 구축하기
3. 특성 엔지니어링 적용하기
4. 다중 모델 비교 (LR, RF, XGBoost)

**Part 3: 최적화 및 리포트**
1. 하이퍼파라미터 튜닝 수행하기
2. 모델 해석 및 특성 중요도 분석하기
3. Plotly 인터랙티브 대시보드 구축하기
4. 비즈니스 인사이트 도출 및 리포트 작성하기

---

## 왜 이것을 배우나요?

| 개념 | 비즈니스 활용 | 실무 예시 |
|------|-------------|----------|
| End-to-End 파이프라인 | 실제 프로젝트 수행 | Kaggle 대회, 실무 ML 시스템 |
| 다중 모델 비교 | 최적 모델 선택 | 성능-해석력 트레이드오프 |
| 특성 엔지니어링 | 모델 성능 향상 | 도메인 지식 반영 |
| 하이퍼파라미터 튜닝 | 모델 최적화 | GridSearch, Optuna |
| 비즈니스 인사이트 | 의사결정 지원 | 경영진 리포트, 대시보드 |

**분석가 관점**: 이번 프로젝트는 Week 4에서 배운 모든 ML 기법을 통합하여 실제 비즈니스 문제를 해결합니다. 고객 이탈 예측은 전자상거래에서 가장 중요한 문제 중 하나로, 이 프로젝트를 통해 포트폴리오 가치가 높은 경험을 쌓게 됩니다!

---

# Part 1: 프로젝트 설계

---

## 1.1 비즈니스 문제 정의

### 프로젝트 배경

**전자상거래 고객 이탈 예측 (Customer Churn Prediction)**

- **비즈니스 상황**: 이커머스 기업의 월간 고객 이탈률이 15%를 넘어 심각한 매출 손실 발생
- **비즈니스 목표**: 이탈 가능 고객을 사전에 예측하여 리텐션 마케팅 타겟팅
- **기대 효과**: 이탈률 20% 감소 시 연간 매출 5억원 증가 예상

In [ ]:
# 프로젝트 정의서
project_definition = {
    "프로젝트명": "전자상거래 고객 이탈 예측 시스템",
    "비즈니스_목표": "이탈 가능 고객 사전 예측 및 리텐션 마케팅 타겟팅",
    "ML_문제_유형": "이진 분류 (Binary Classification)",
    "타겟_변수": "churned (0: 유지, 1: 이탈)",
    "평가_지표": ["AUC-ROC (주요)", "Precision", "Recall", "F1 Score"],
    "성공_기준": "AUC-ROC >= 0.80",
    "데이터셋": "ecommerce_churn.csv (200명 고객)"
}

print("=" * 60)
print("ML 프로젝트 정의서")
print("=" * 60)
for key, value in project_definition.items():
    print(f"{key}: {value}")

### 평가 지표 선택 이유

| 지표 | 선택 이유 |
|------|----------|
| AUC-ROC | 클래스 불균형 상황에서 모델의 전반적인 성능 평가 |
| Precision | 이탈 예측 시 실제 이탈 고객 비율 (마케팅 비용 효율) |
| Recall | 실제 이탈 고객 중 탐지 비율 (이탈 고객 놓치지 않기) |
| F1 Score | Precision과 Recall의 조화 평균 |

---

## 1.2 데이터셋 로드 및 탐색

In [ ]:
# 필수 라이브러리 임포트
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# scikit-learn
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, confusion_matrix, classification_report
)
from sklearn.pipeline import Pipeline

# XGBoost
try:
    from xgboost import XGBClassifier
    XGBOOST_AVAILABLE = True
except ImportError:
    print("XGBoost not installed. Install with: pip install xgboost")
    XGBOOST_AVAILABLE = False

import warnings
warnings.filterwarnings('ignore')

print("라이브러리 로드 완료!")

In [ ]:
# 데이터 로드
df = pd.read_csv('datasets/ecommerce_churn.csv')

print(f"데이터 크기: {df.shape[0]}행 x {df.shape[1]}열")
print(f"\n컬럼 목록:")
print(df.columns.tolist())

In [ ]:
# 데이터 미리보기
df.head(10)

In [ ]:
# 데이터 기본 정보
df.info()

In [ ]:
# 기술 통계량
df.describe()

### 데이터 딕셔너리

| 컬럼명 | 설명 | 타입 |
|--------|------|------|
| customer_id | 고객 고유 ID | 식별자 |
| gender | 성별 (M/F) | 범주형 |
| age | 나이 | 수치형 |
| tenure_months | 가입 기간 (월) | 수치형 |
| contract_type | 계약 유형 (Monthly/Quarterly/Annual) | 범주형 |
| monthly_charge | 월 결제 금액 | 수치형 |
| total_charge | 총 결제 금액 | 수치형 |
| payment_method | 결제 수단 | 범주형 |
| num_products | 구매 상품 수 | 수치형 |
| complaints | 불만 접수 횟수 | 수치형 |
| support_calls | 고객센터 문의 횟수 | 수치형 |
| last_purchase_days | 마지막 구매 후 경과 일수 | 수치형 |
| avg_order_value | 평균 주문 금액 | 수치형 |
| order_frequency | 주문 빈도 (월) | 수치형 |
| satisfaction_score | 만족도 점수 (1-5) | 수치형 |
| **churned** | **이탈 여부 (0/1)** | **타겟** |

In [ ]:
# 타겟 변수 분포 확인
churn_counts = df['churned'].value_counts()
churn_pct = df['churned'].value_counts(normalize=True) * 100

print("타겟 변수 분포:")
print(f"  유지 (0): {churn_counts[0]}명 ({churn_pct[0]:.1f}%)")
print(f"  이탈 (1): {churn_counts[1]}명 ({churn_pct[1]:.1f}%)")
print(f"\n클래스 불균형 비율: 1:{churn_counts[0]/churn_counts[1]:.1f}")

---

## 1.3 결측치 및 데이터 품질 확인

In [ ]:
# 결측치 확인
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)

missing_df = pd.DataFrame({
    '결측치 수': missing,
    '비율(%)': missing_pct
})

print("결측치 현황:")
print(missing_df[missing_df['결측치 수'] > 0] if missing.sum() > 0 else "결측치 없음!")

In [ ]:
# 중복 데이터 확인
duplicates = df.duplicated().sum()
print(f"중복 행 수: {duplicates}")

---

# Part 2: EDA 및 데이터 전처리

---

## 2.1 탐색적 데이터 분석 (EDA)

In [ ]:
# 타겟 분포 시각화
fig = px.pie(
    values=churn_counts.values,
    names=['유지', '이탈'],
    title='고객 이탈 분포',
    color_discrete_sequence=['#2ecc71', '#e74c3c'],
    hole=0.4
)
fig.update_traces(textinfo='percent+label+value')
fig.show()

In [ ]:
# 수치형 변수 분포 (이탈 여부별)
numeric_cols = ['age', 'tenure_months', 'monthly_charge', 'complaints', 
                'support_calls', 'last_purchase_days', 'satisfaction_score']

fig = make_subplots(rows=2, cols=4, subplot_titles=numeric_cols + [''])

for i, col in enumerate(numeric_cols):
    row = i // 4 + 1
    col_idx = i % 4 + 1
    
    for churned, color, name in [(0, '#2ecc71', '유지'), (1, '#e74c3c', '이탈')]:
        fig.add_trace(
            go.Histogram(
                x=df[df['churned'] == churned][col],
                name=name,
                opacity=0.7,
                marker_color=color,
                showlegend=(i == 0)
            ),
            row=row, col=col_idx
        )

fig.update_layout(height=500, title_text="수치형 변수 분포 (이탈 여부별)", barmode='overlay')
fig.show()

In [ ]:
# 계약 유형별 이탈률
contract_churn = df.groupby('contract_type')['churned'].agg(['sum', 'count'])
contract_churn['churn_rate'] = (contract_churn['sum'] / contract_churn['count'] * 100).round(1)
contract_churn = contract_churn.reset_index()

fig = px.bar(
    contract_churn,
    x='contract_type',
    y='churn_rate',
    title='계약 유형별 이탈률',
    labels={'contract_type': '계약 유형', 'churn_rate': '이탈률 (%)'},
    color='churn_rate',
    color_continuous_scale='RdYlGn_r',
    text='churn_rate'
)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.show()

In [ ]:
# 만족도 점수별 이탈률
satisfaction_churn = df.groupby('satisfaction_score')['churned'].mean() * 100

fig = px.bar(
    x=satisfaction_churn.index,
    y=satisfaction_churn.values,
    title='만족도 점수별 이탈률',
    labels={'x': '만족도 점수', 'y': '이탈률 (%)'},
    color=satisfaction_churn.values,
    color_continuous_scale='RdYlGn_r'
)
fig.show()

In [ ]:
# 상관관계 분석
numeric_features = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_features.remove('churned')  # 타겟 제외

# 타겟과의 상관관계
correlations = df[numeric_features + ['churned']].corr()['churned'].drop('churned').sort_values()

fig = px.bar(
    x=correlations.values,
    y=correlations.index,
    orientation='h',
    title='특성과 이탈의 상관관계',
    labels={'x': '상관계수', 'y': '특성'},
    color=correlations.values,
    color_continuous_scale='RdBu_r'
)
fig.show()

### EDA 인사이트

1. **계약 유형**: Monthly 계약자의 이탈률이 가장 높음 (단기 계약 = 높은 이탈 위험)
2. **만족도**: 만족도 1-2점 고객의 이탈률이 매우 높음
3. **불만/문의**: 불만 건수, 고객센터 문의 횟수가 많을수록 이탈 가능성 증가
4. **마지막 구매**: 마지막 구매 후 경과일이 길수록 이탈 위험 증가
5. **가입 기간**: 장기 가입자일수록 이탈률 낮음 (충성 고객)

---

## 2.2 특성 엔지니어링

In [ ]:
# 특성 엔지니어링
df_fe = df.copy()

# 1. 고객 가치 점수 (tenure, total_charge, order_frequency 기반)
df_fe['customer_value'] = (
    df_fe['tenure_months'] / 60 + 
    df_fe['total_charge'] / df_fe['total_charge'].max() + 
    df_fe['order_frequency'] / df_fe['order_frequency'].max()
) / 3

# 2. 문제 고객 지표 (불만 + 고객센터 문의 합산)
df_fe['problem_score'] = df_fe['complaints'] + df_fe['support_calls']

# 3. 구매 활성도 (최근 구매일 역수)
df_fe['purchase_recency_score'] = 1 / (df_fe['last_purchase_days'] + 1)

# 4. 월간 충전 대비 주문 가치 비율
df_fe['value_ratio'] = df_fe['avg_order_value'] / (df_fe['monthly_charge'] + 1)

# 5. 계약 유형 리스크 점수
contract_risk = {'Monthly': 3, 'Quarterly': 2, 'Annual': 1}
df_fe['contract_risk'] = df_fe['contract_type'].map(contract_risk)

print("생성된 특성:")
new_features = ['customer_value', 'problem_score', 'purchase_recency_score', 'value_ratio', 'contract_risk']
print(df_fe[new_features].describe())

In [ ]:
# 범주형 변수 인코딩
# gender: Label Encoding
df_fe['gender_encoded'] = (df_fe['gender'] == 'M').astype(int)

# payment_method: Label Encoding
le_payment = LabelEncoder()
df_fe['payment_encoded'] = le_payment.fit_transform(df_fe['payment_method'])

# contract_type: One-Hot Encoding
contract_dummies = pd.get_dummies(df_fe['contract_type'], prefix='contract')
df_fe = pd.concat([df_fe, contract_dummies], axis=1)

print("인코딩 완료!")
print(f"최종 특성 수: {df_fe.shape[1]}개")

---

## 2.3 데이터 분할 및 전처리 파이프라인

In [ ]:
# 최종 특성 선택
feature_cols = [
    # 원본 수치형 특성
    'age', 'tenure_months', 'monthly_charge', 'total_charge',
    'num_products', 'complaints', 'support_calls', 'last_purchase_days',
    'avg_order_value', 'order_frequency', 'satisfaction_score',
    # 엔지니어링 특성
    'customer_value', 'problem_score', 'purchase_recency_score', 
    'value_ratio', 'contract_risk',
    # 인코딩 특성
    'gender_encoded', 'payment_encoded',
    'contract_Annual', 'contract_Monthly', 'contract_Quarterly'
]

X = df_fe[feature_cols]
y = df_fe['churned']

print(f"특성 행렬 크기: {X.shape}")
print(f"타겟 벡터 크기: {y.shape}")

In [ ]:
# Train/Test 분할 (stratify로 클래스 비율 유지)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,
    stratify=y
)

print(f"훈련 데이터: {X_train.shape[0]}개")
print(f"테스트 데이터: {X_test.shape[0]}개")
print(f"\n훈련 데이터 이탈률: {y_train.mean():.1%}")
print(f"테스트 데이터 이탈률: {y_test.mean():.1%}")

In [ ]:
# 스케일링 (StandardScaler)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("스케일링 완료!")
print(f"스케일링 후 훈련 데이터 평균: {X_train_scaled.mean():.4f}")
print(f"스케일링 후 훈련 데이터 표준편차: {X_train_scaled.std():.4f}")

---

# Part 3: 모델 훈련 및 비교

---

## 3.1 베이스라인 모델: Logistic Regression

In [ ]:
# Logistic Regression
lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(X_train_scaled, y_train)

# 예측
y_pred_lr = lr_model.predict(X_test_scaled)
y_proba_lr = lr_model.predict_proba(X_test_scaled)[:, 1]

# 평가
print("Logistic Regression 결과")
print("=" * 40)
print(f"Accuracy: {accuracy_score(y_test, y_pred_lr):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_lr):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_lr):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_lr):.4f}")
print(f"AUC-ROC: {roc_auc_score(y_test, y_proba_lr):.4f}")

---

## 3.2 Random Forest

In [ ]:
# Random Forest
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_train_scaled, y_train)

# 예측
y_pred_rf = rf_model.predict(X_test_scaled)
y_proba_rf = rf_model.predict_proba(X_test_scaled)[:, 1]

# 평가
print("Random Forest 결과")
print("=" * 40)
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_rf):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_rf):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_rf):.4f}")
print(f"AUC-ROC: {roc_auc_score(y_test, y_proba_rf):.4f}")

---

## 3.3 XGBoost

In [ ]:
# XGBoost
if XGBOOST_AVAILABLE:
    xgb_model = XGBClassifier(
        n_estimators=100,
        max_depth=5,
        learning_rate=0.1,
        random_state=42,
        use_label_encoder=False,
        eval_metric='logloss'
    )
    xgb_model.fit(X_train_scaled, y_train)
    
    # 예측
    y_pred_xgb = xgb_model.predict(X_test_scaled)
    y_proba_xgb = xgb_model.predict_proba(X_test_scaled)[:, 1]
    
    # 평가
    print("XGBoost 결과")
    print("=" * 40)
    print(f"Accuracy: {accuracy_score(y_test, y_pred_xgb):.4f}")
    print(f"Precision: {precision_score(y_test, y_pred_xgb):.4f}")
    print(f"Recall: {recall_score(y_test, y_pred_xgb):.4f}")
    print(f"F1 Score: {f1_score(y_test, y_pred_xgb):.4f}")
    print(f"AUC-ROC: {roc_auc_score(y_test, y_proba_xgb):.4f}")
else:
    print("XGBoost not available. Skipping...")

---

## 3.4 모델 성능 비교

In [ ]:
# 모델 성능 비교 표
results = {
    'Model': ['Logistic Regression', 'Random Forest'],
    'Accuracy': [accuracy_score(y_test, y_pred_lr), accuracy_score(y_test, y_pred_rf)],
    'Precision': [precision_score(y_test, y_pred_lr), precision_score(y_test, y_pred_rf)],
    'Recall': [recall_score(y_test, y_pred_lr), recall_score(y_test, y_pred_rf)],
    'F1': [f1_score(y_test, y_pred_lr), f1_score(y_test, y_pred_rf)],
    'AUC-ROC': [roc_auc_score(y_test, y_proba_lr), roc_auc_score(y_test, y_proba_rf)]
}

if XGBOOST_AVAILABLE:
    results['Model'].append('XGBoost')
    results['Accuracy'].append(accuracy_score(y_test, y_pred_xgb))
    results['Precision'].append(precision_score(y_test, y_pred_xgb))
    results['Recall'].append(recall_score(y_test, y_pred_xgb))
    results['F1'].append(f1_score(y_test, y_pred_xgb))
    results['AUC-ROC'].append(roc_auc_score(y_test, y_proba_xgb))

results_df = pd.DataFrame(results)
results_df = results_df.round(4)
print("모델 성능 비교")
print(results_df.to_string(index=False))

In [ ]:
# 성능 비교 시각화
fig = go.Figure()

metrics = ['Accuracy', 'Precision', 'Recall', 'F1', 'AUC-ROC']
colors = ['#3498db', '#2ecc71', '#e74c3c']

for i, model in enumerate(results_df['Model']):
    fig.add_trace(go.Bar(
        name=model,
        x=metrics,
        y=[results_df.loc[i, m] for m in metrics],
        marker_color=colors[i]
    ))

fig.update_layout(
    title='모델 성능 비교',
    barmode='group',
    yaxis_title='Score',
    yaxis=dict(range=[0, 1.1])
)
fig.show()

In [ ]:
# ROC 커브 비교
fig = go.Figure()

# Logistic Regression
fpr_lr, tpr_lr, _ = roc_curve(y_test, y_proba_lr)
fig.add_trace(go.Scatter(x=fpr_lr, y=tpr_lr, mode='lines', 
                         name=f'LR (AUC={roc_auc_score(y_test, y_proba_lr):.3f})'))

# Random Forest
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_proba_rf)
fig.add_trace(go.Scatter(x=fpr_rf, y=tpr_rf, mode='lines',
                         name=f'RF (AUC={roc_auc_score(y_test, y_proba_rf):.3f})'))

# XGBoost
if XGBOOST_AVAILABLE:
    fpr_xgb, tpr_xgb, _ = roc_curve(y_test, y_proba_xgb)
    fig.add_trace(go.Scatter(x=fpr_xgb, y=tpr_xgb, mode='lines',
                             name=f'XGB (AUC={roc_auc_score(y_test, y_proba_xgb):.3f})'))

# 대각선 (Random)
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', 
                         line=dict(dash='dash', color='gray'), name='Random'))

fig.update_layout(
    title='ROC Curve 비교',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate'
)
fig.show()

---

# Part 4: 하이퍼파라미터 튜닝 및 최종 모델

---

## 4.1 GridSearchCV를 이용한 튜닝

In [ ]:
# Random Forest 하이퍼파라미터 튜닝
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train_scaled, y_train)

print(f"\n최적 파라미터: {grid_search.best_params_}")
print(f"최적 AUC-ROC (CV): {grid_search.best_score_:.4f}")

In [ ]:
# 최적 모델로 최종 평가
best_model = grid_search.best_estimator_

y_pred_best = best_model.predict(X_test_scaled)
y_proba_best = best_model.predict_proba(X_test_scaled)[:, 1]

print("최적 Random Forest 모델 결과")
print("=" * 40)
print(f"Accuracy: {accuracy_score(y_test, y_pred_best):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_best):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_best):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_best):.4f}")
print(f"AUC-ROC: {roc_auc_score(y_test, y_proba_best):.4f}")

---

## 4.2 특성 중요도 분석

In [ ]:
# 특성 중요도
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': best_model.feature_importances_
}).sort_values('importance', ascending=True)

# 시각화
fig = px.bar(
    feature_importance.tail(15),
    x='importance',
    y='feature',
    orientation='h',
    title='특성 중요도 (Top 15)',
    labels={'importance': '중요도', 'feature': '특성'},
    color='importance',
    color_continuous_scale='Blues'
)
fig.show()

In [ ]:
# 혼동 행렬
cm = confusion_matrix(y_test, y_pred_best)

fig = px.imshow(
    cm,
    labels=dict(x="예측", y="실제", color="Count"),
    x=['유지', '이탈'],
    y=['유지', '이탈'],
    text_auto=True,
    title='혼동 행렬 (Confusion Matrix)',
    color_continuous_scale='Blues'
)
fig.show()

---

# Part 5: 비즈니스 인사이트 및 대시보드

---

## 5.1 이탈 위험 고객 식별

In [ ]:
# 전체 데이터에 대한 이탈 확률 예측
X_all_scaled = scaler.transform(X)
df_fe['churn_probability'] = best_model.predict_proba(X_all_scaled)[:, 1]

# 위험 등급 분류
def risk_grade(prob):
    if prob >= 0.7:
        return '고위험'
    elif prob >= 0.4:
        return '중위험'
    else:
        return '저위험'

df_fe['risk_grade'] = df_fe['churn_probability'].apply(risk_grade)

# 위험 등급별 분포
risk_dist = df_fe['risk_grade'].value_counts()
print("위험 등급별 고객 분포:")
print(risk_dist)

In [ ]:
# 고위험 고객 리스트
high_risk_customers = df_fe[df_fe['risk_grade'] == '고위험'][[
    'customer_id', 'age', 'tenure_months', 'contract_type',
    'satisfaction_score', 'complaints', 'churn_probability'
]].sort_values('churn_probability', ascending=False)

print(f"고위험 고객 수: {len(high_risk_customers)}명")
print("\n고위험 고객 목록 (상위 10명):")
high_risk_customers.head(10)

---

## 5.2 비즈니스 인사이트 요약

In [ ]:
# 비즈니스 인사이트 대시보드
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        '위험 등급별 분포',
        '계약 유형별 평균 이탈 확률',
        '만족도별 평균 이탈 확률',
        '가입 기간별 이탈 확률'
    ],
    specs=[[{"type": "pie"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "scatter"}]]
)

# 1. 위험 등급 파이 차트
fig.add_trace(
    go.Pie(labels=risk_dist.index, values=risk_dist.values,
           marker_colors=['#e74c3c', '#f39c12', '#2ecc71']),
    row=1, col=1
)

# 2. 계약 유형별 이탈 확률
contract_prob = df_fe.groupby('contract_type')['churn_probability'].mean().sort_values(ascending=False)
fig.add_trace(
    go.Bar(x=contract_prob.index, y=contract_prob.values, marker_color='#3498db'),
    row=1, col=2
)

# 3. 만족도별 이탈 확률
sat_prob = df_fe.groupby('satisfaction_score')['churn_probability'].mean()
fig.add_trace(
    go.Bar(x=sat_prob.index, y=sat_prob.values, marker_color='#9b59b6'),
    row=2, col=1
)

# 4. 가입 기간별 이탈 확률 (산점도)
fig.add_trace(
    go.Scatter(x=df_fe['tenure_months'], y=df_fe['churn_probability'],
               mode='markers', marker=dict(color='#1abc9c', opacity=0.5)),
    row=2, col=2
)

fig.update_layout(height=700, title_text="고객 이탈 인사이트 대시보드", showlegend=False)
fig.show()

In [ ]:
# 최종 비즈니스 리포트
print("="*60)
print("고객 이탈 예측 프로젝트 - 최종 리포트")
print("="*60)

print("\n[1] 프로젝트 개요")
print("-" * 40)
print(f"  - 분석 대상: {len(df)}명 고객")
print(f"  - 현재 이탈률: {df['churned'].mean():.1%}")
print(f"  - 사용 모델: Random Forest (GridSearch 튜닝)")

print("\n[2] 모델 성능")
print("-" * 40)
print(f"  - AUC-ROC: {roc_auc_score(y_test, y_proba_best):.4f}")
print(f"  - Precision: {precision_score(y_test, y_pred_best):.4f}")
print(f"  - Recall: {recall_score(y_test, y_pred_best):.4f}")

print("\n[3] 핵심 이탈 요인 (Top 5)")
print("-" * 40)
top_features = feature_importance.tail(5)
for idx, row in top_features.iterrows():
    print(f"  - {row['feature']}: {row['importance']:.4f}")

print("\n[4] 위험 고객 현황")
print("-" * 40)
print(f"  - 고위험: {len(df_fe[df_fe['risk_grade'] == '고위험'])}명")
print(f"  - 중위험: {len(df_fe[df_fe['risk_grade'] == '중위험'])}명")
print(f"  - 저위험: {len(df_fe[df_fe['risk_grade'] == '저위험'])}명")

print("\n[5] 권장 조치")
print("-" * 40)
print("  1. 고위험 고객 대상 즉시 리텐션 캠페인 실행")
print("  2. Monthly 계약자 → Annual 전환 프로모션")
print("  3. 만족도 1-2점 고객 대상 CS 개선")
print("  4. 불만 3건 이상 고객 VIP 담당자 배정")
print("  5. 60일 이상 미구매 고객 쿠폰 발송")

print("\n" + "="*60)

---

## 실습 퀴즈

**난이도**: (쉬움) ~ (어려움)

---

### Q1. 프로젝트 정의 작성

**문제**: 아래 빈칸을 채워 "중고차 가격 예측" 프로젝트 정의서를 완성하세요.

```python
project = {
    "프로젝트명": "중고차 가격 예측 시스템",
    "ML_문제_유형": ___,  # 분류 or 회귀?
    "평가_지표": ___,     # 적절한 지표?
    "성공_기준": ___      # 어떤 기준?
}
```

In [ ]:
# 여기에 코드를 작성하세요
project = {
    "프로젝트명": "중고차 가격 예측 시스템",
    "ML_문제_유형": None,  # 채우세요
    "평가_지표": None,     # 채우세요
    "성공_기준": None      # 채우세요
}


### Q2. 결측치 확인

**문제**: df 데이터프레임의 결측치를 컬럼별로 확인하고, 결측치가 있는 컬럼만 출력하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q3. 클래스 불균형 확인

**문제**: 타겟 변수 'churned'의 클래스별 비율을 계산하고, 불균형 비율을 출력하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q4. 새로운 특성 생성

**문제**: 'monthly_charge'와 'tenure_months'를 사용하여 '예상 총 결제액' 특성을 생성하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q5. 데이터 분할

**문제**: X, y를 train/test로 분할하세요 (test_size=0.25, stratify 적용, random_state=42).

In [ ]:
# 여기에 코드를 작성하세요


### Q6. Logistic Regression 훈련

**문제**: StandardScaler + LogisticRegression 파이프라인을 구성하고 훈련하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q7. 모델 평가

**문제**: 훈련된 모델의 Accuracy, Precision, Recall, F1, AUC-ROC를 계산하여 출력하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q8. 혼동 행렬 시각화

**문제**: 테스트 데이터에 대한 혼동 행렬을 Plotly로 시각화하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q9. GridSearchCV 튜닝

**문제**: RandomForest에 대해 다음 파라미터로 GridSearchCV를 수행하세요.
- n_estimators: [50, 100]
- max_depth: [5, 10]
- scoring: 'roc_auc'
- cv: 3

In [ ]:
# 여기에 코드를 작성하세요


### Q10. 비즈니스 인사이트 도출

**문제**: 최종 모델을 사용하여 다음을 수행하세요.
1. 전체 고객에 대한 이탈 확률 예측
2. 이탈 확률 0.5 이상인 고객 수 계산
3. 상위 5개 특성 중요도 출력
4. 비즈니스 권장 조치 1개 작성

In [ ]:
# 여기에 코드를 작성하세요


---

## 학습 정리

### Part 1: 프로젝트 설계 핵심 요약

| 단계 | 핵심 활동 | 결과물 |
|-----|----------|--------|
| 문제 정의 | 비즈니스 목표 -> ML 문제 변환 | 프로젝트 정의서 |
| 데이터 탐색 | 결측치, 분포, 상관관계 확인 | EDA 리포트 |
| 특성 설계 | 도메인 지식 기반 특성 생성 | 특성 엔지니어링 계획 |

### Part 2: 모델링 핵심 요약

| 모델 | 장점 | 단점 | 사용 시기 |
|-----|------|------|----------|
| Logistic Regression | 해석 용이, 빠름 | 비선형 학습 어려움 | 베이스라인, 해석 중요 |
| Random Forest | 과적합 방지, 강건 | 느린 예측 | 대부분의 정형 데이터 |
| XGBoost | 높은 성능 | 튜닝 필요 | 최고 성능 필요시 |

### Part 3: 최적화 및 해석 핵심 요약

| 기법 | 목적 | 주요 메서드 |
|-----|------|------------|
| GridSearchCV | 하이퍼파라미터 튜닝 | fit, best_params_, best_score_ |
| 특성 중요도 | 영향력 분석 | feature_importances_ |
| 혼동 행렬 | 오분류 패턴 분석 | confusion_matrix |

### 실무 팁

1. **베이스라인 먼저**: 복잡한 모델 전 간단한 모델로 기준선 설정
2. **특성 엔지니어링**: 도메인 지식 활용이 모델 성능에 큰 영향
3. **불균형 처리**: stratify, class_weight, SMOTE 등 고려
4. **비즈니스 연결**: 모델 결과를 비즈니스 액션으로 연결
5. **시각화 활용**: Plotly로 인터랙티브 대시보드 구축